<a href="https://colab.research.google.com/github/Zezoo123/DeFi-Arbitrage-and-Prediction-Tool/blob/nn_implementation/neural_networks/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [9]:
folderpath = '/content/drive/MyDrive/BTC/'

df = pd.read_csv(folderpath + '15m.csv')
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,2024-09-17 18:30:00,61001.53,61040.00,60848.48,60857.36,150.23181,2024-09-17 18:44:59.999,9.157295e+06,25688,62.55571,3.812885e+06,0
1,2024-09-17 18:45:00,60857.37,60900.00,60363.81,60540.17,692.43883,2024-09-17 18:59:59.999,4.194843e+07,66688,308.05039,1.865586e+07,0
2,2024-09-17 19:00:00,60540.17,60696.48,60482.00,60647.56,352.39735,2024-09-17 19:14:59.999,2.134187e+07,58292,166.95392,1.011010e+07,0
3,2024-09-17 19:15:00,60647.56,60647.56,60312.00,60420.00,250.44923,2024-09-17 19:29:59.999,1.513811e+07,57333,88.01325,5.318536e+06,0
4,2024-09-17 19:30:00,60419.99,60448.00,60132.56,60132.56,442.55572,2024-09-17 19:44:59.999,2.667157e+07,64595,151.52786,9.132072e+06,0


In [17]:
# Preprocess data
df_processed = df[['Open', 'High', 'Low', 'Close', 'Volume']]

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df_processed)
df_processed = pd.DataFrame(scaled_data, columns=df_processed.columns)
df_processed.head()

,Open,High,Low,Close,Volume
0,0.209359,0.198839,0.207420,0.193642,0.034489
1,0.193643,0.183649,0.154578,0.159063,0.176655
2,0.159063,0.161567,0.167464,0.170770,0.087497
3,0.170770,0.156259,0.148930,0.145962,0.060766
4,0.145961,0.134607,0.129366,0.114627,0.111136


In [27]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(data, window_size=5):
  X = []
  y = []
  close_prices = data['Close']
  for i in range(len(data) - window_size):
    X.append(data.iloc[i: i + window_size].values)
    y.append(close_prices[i + window_size]) # Close Value
  return np.array(X), np.array(y)

In [32]:
WINDOW_SIZE = 5

X, y = df_to_X_y(df_processed, window_size=WINDOW_SIZE)
X.shape, y.shape

((2871, 5, 5), (2871,))

In [35]:
#Split the data
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

In [39]:
#Create LSTM model
model = tf.keras.Sequential(([
    tf.keras.layers.InputLayer(shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)  # Predict the 'Close' price
]))

#use adam optimizer
#loss calculated using mean squared error
model.compile(optimizer='adam', loss='mse')

(5, 5)